In [1]:
from pathlib import Path 
import tensorflow as tf
from keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [ ]:
PROJECT_PATH = Path.cwd().parent
MODELS_DIRPATH = PROJECT_PATH / 'models'

PROJECT_PATH, MODELS_DIRPATH

## 1. Load the Trained Model, Scalers, and Encoders

In [ ]:
# Load the trained model
model = load_model(MODELS_DIRPATH / 'best_model.keras')

# Load encoders and  scalers
with open(MODELS_DIRPATH / 'onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open(MODELS_DIRPATH / 'label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open(MODELS_DIRPATH / 'scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

## 2. Encode and Scale The Input Data

In [4]:
# Example input data
input_data = {
    'CreditScore': 647,
    'Geography': 'Spain',
    'Gender': 'Female',
    'Age': 44,
    'Tenure': 5,
    'Balance': 0,
    'NumOfProducts': 3,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 174205.22,
}

In [ ]:
input_df = pd.DataFrame([input_data])
input_df

### 2.1 Encode Geography with One-Hot Encoder

In [ ]:
# One-Hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

### 2.2 Encode Gender with Label Encoder

In [ ]:
# Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

### 2.3 Concatenate one-hot encoded data

In [ ]:
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df


### 2.4 Scaling the Input Data

In [ ]:
input_scaled = scaler.transform(input_df)
input_scaled

## 3. Predict Banking Churn

In [ ]:
prediction = model.predict(input_scaled)
prediction

In [ ]:
prediction_proba = prediction[0][0]
prediction_proba

In [ ]:
if prediction_proba > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")